In [1]:
P = Primes()
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
C.<x> = PolynomialRing(CC)
S = PowerSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m,x):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m,x):
    exn = exNo3c(n+2,m,x)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m,x):
    f = J(n,m,x)
    return f.truncate(n+1)
def Jcoefficient(n,m,x):
    f = J(n+2,m,x)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m,x): 
    f=J(n,m,x)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m,x):
    return xjNewStep(n-1,m,x)
def xjNewCoefficient(n,m,x):
    if n>0:return (xjNew(n,m,x)-xjNew(n-1,m,x))/x^n
    if n==0:return 1
def xjNewTest(n,m,x):
    return D(xjNewStep(n-1,m,x))
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def prime_n(a):return P.unrank(a-1)

def primeFactors(n):
    answer=[]
    lf=list(n.factor())
    length=len(lf)
    for k in [0..length-1]:
        pair=lf[k]
        (base,exponent)=pair
        answer=answer+[[base]]
    return(flatten(answer))

def primeFactorsInList(lst):
    ld=len(lst)
    primelist=[]
    for k in [0..ld-1]:
        a=ZZ(lst[k])
        primelist=primelist+[primeFactors(a)]
    primelist=flatten(primelist)
    primelist=list(set(primelist))
    return primelist

def den(a):
    if a in QQ:
        if a in ZZ:return 1
        return a.denominator()
def listdens(lst):
    return list(den(a) for a in lst)
def polynomialdenominator(poly):return lcm(listdens(poly))
def polynomialnumerator(poly):
    return expand(poly*polynomialdenominator(poly))
def num(a):return a.numerator()
def prime(n):
    P=Primes()
    return P.unrank(n-1)
def inclusiveListPrimesTo(n):
    if is_prime(n):
        return list(primes(n+1))
    else:
        return list(primes(n))
def chunk(k,list):
    chnk=[]
    for n in [k..len(list)-1]:
        chnk=chnk+[list[n]]
    return chnk
def drop(m,lst): return list(Set(lst).difference({m}))
def sort(list):
    answr=[]
    chnk=list
    for n in [0..len(list)-1]:
        m = min(chnk)
        answr=answr+[m]
        chnk=drop(m,chnk)
    return answr
def ord(p,n):return QQ.valuation(p)(n)
def base_conversion(number, base_from, base_to):  # by Vishnu Namboothiri
# https://ask.sagemath.org/question/9657/how-to-output-decimal-numbers-in-different-radix/
    if(base_from==10):
        return stripQuotationMarks((number.str(base=base_to)))
    else:
        decimal = Integer(str(number), base=base_from)
        return (stripQuotationMarks(decimal.str(base=base_to)))
def lastFactorWithIntegralFactors(polynomial):
    return polynomialnumerator(lastfactor(polynomial))
def numericalFactorWithIntegralFactors(polynomial):
    nf=numericalfactor(polynomial);
    pd=polynomialdenominator(lastfactor(polynomial))
    return nf/pd


def primeDivisors(n):
    pd=[]
    n=n
    fn=factor(n)
    ln=len(fn)
    for k in [0..ln-1]:
        part = fn[k]
        (prime,exponent) = part
        pd = pd + [prime]
    return pd 
def fOverField(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return ppoly

###################### following two routines disagree ####################
def factorOverField(f,prime,power):
    answer=[]
    T = GF(prime^power,'t')
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return fp

def factorOverField2(poly,prime,power,j):
    sd = leastSplittingDegree(poly,prime,bound)
    T=GF(prime^j)
    G.<t>= GF(prime^sd)
    U.<x> = T[]
    ppoly=U(poly)
    fpp=ppoly.factor()
    return fpp

############################

#######################################################
# splitting tests below fail at  e.g. f = x + 1 
############################################################

def splitsTF(f,prime,power):
    T = GF(prime^power)
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx > 1:return(False)
    return(True)

def leastSplittingDegree(f,prime,bound):
    for power in [1..bound]:
        tf=splitsTF(f,prime,power)
        if tf == True:return power
    return -1

#########################################################

def firstNprimes(n):
    P=Primes()
    first=[]
    for k in [0..n-1]:
        first=first+[P.unrank(k)]
    return first

def slopeExistsTF(pairlist):
    lenth=len(pairlist)
    if lenth<3:return False
    slopes=[]
    for k in [1..lenth-1]:
        pair1=pairlist[k-1]
        abscissa1=pair1[0]
        ordinate1=pair1[1]
        pair2=pairlist[k]
        abscissa2=pair2[0]
        ordinate2=pair2[1]
        dx=abscissa2-abscissa1
        if dx == 0:return False
        dy=ordinate2-ordinate1
        slope=dy/dx
        slopes=slopes+[slope]
    slopes=Set(slopes)
    if len(slopes) == 1:
        return [True,slopes[0]]
    if len(slopes) > 1:
        return False 
    
def zeroOfLinearMonomial(monomial,x):return x-monomial

def totalDegreeOverField(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    td=0
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        td=td+exponent
    return td

def logsAndMultiplicitiesOverFieldTest(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    print gf
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data



def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data

def padListOnRightToLengthK(listo,k):
    length=len(listo)
    padded=listo
   
    if length<k:
        for j in [length..k-1]:
            padded=padded+[0]
           
        return padded
    return listo
            

In [3]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print len(s)
print s[1]

201
[0, 3/8*x^2 + 1/2]


In [ ]:
## printed output has been snipped

import pickle
wfile=open('/Users/barrybrent/run1jun21no1.txt','a')
data=[]
for m in [3..500]:
    print m
    data=data+[[m,normalJ(200,m)]]
s=pickle.dumps(str(data))
wfile.write(s)
wfile.close()
#########################
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

polydata=[]

wfile = open('/Users/barrybrent/run14jun21no14.txt','a') # xJ polynomials
for n in [0..200]:
    data=[]
    for k in [0..2*n+1]:
        print (n,k)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n-1,rl]] 

    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close() # print-output snipped

In [2]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=7
residue=0
bound=120
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        zerodata=[]
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        order=prime^sd
        td=totalDegreeOverField(poly,prime,sd)
        lm=logsAndMultiplicitiesOverFieldTest(poly,prime,sd)
        logs=[]
        digits=[]
        multiplicities=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            multiplicity=lm[j][1]
            digit=log.digits(prime)
            digit= padListOnRightToLengthK(digit,sd)
            logs=logs+[log]
            multiplicities=multiplicities+[multiplicity]
            digits=digits+[digit]
        print
        print (index,sd,td,multiplicities)
        print
        print logs;print;print digits
        print "--------------------------------------------------------------------------------------"

(3) * (x + 1) * (x + 6)

(0, 1, 2, [1, 1])

[3, 6]

[[3], [6]]
--------------------------------------------------------------------------------------
(3) * (x + t^3 + 3*t^2 + 6*t) * (x + 3*t^3 + 2*t^2 + 3*t) * (x + 3*t^3 + 6*t^2 + 3) * (x + 4*t^3 + t^2 + 4) * (x + 4*t^3 + 5*t^2 + 4*t) * (x + 6*t^3 + 4*t^2 + t) * (x + 2)^2 * (x + 5)^2 * (x + 3)^3 * (x + 4)^3

(7, 4, 16, [1, 1, 1, 1, 1, 1, 2, 2, 3, 3])

[1525, 600, 2275, 1075, 1800, 325, 2000, 800, 1600, 400]

[[6, 0, 3, 4], [5, 1, 5, 1], [0, 3, 4, 6], [4, 6, 0, 3], [1, 5, 1, 5], [3, 4, 6, 0], [5, 5, 5, 5], [2, 2, 2, 2], [4, 4, 4, 4], [1, 1, 1, 1]]
--------------------------------------------------------------------------------------
(4) * (x + t^3 + 3*t^2 + 6*t) * (x + 3*t^3 + 2*t^2 + 3*t) * (x + 3*t^3 + 6*t^2 + 3) * (x + 4*t^3 + t^2 + 4) * (x + 4*t^3 + 5*t^2 + 4*t) * (x + 6*t^3 + 4*t^2 + t) * (x + 1)^2 * (x + 6)^2 * (x + 2)^4 * (x + 5)^4 * (x + 3)^5 * (x + 4)^5

(14, 4, 28, [1, 1, 1, 1, 1, 1, 2, 2, 4, 4, 5, 5])

[1525, 600, 2275, 1075,


(98, 4, 172, [1, 1, 1, 1, 1, 1, 26, 26, 28, 28, 29, 29])

[1525, 600, 2275, 1075, 1800, 325, 1200, 2400, 2000, 800, 1600, 400]

[[6, 0, 3, 4], [5, 1, 5, 1], [0, 3, 4, 6], [4, 6, 0, 3], [1, 5, 1, 5], [3, 4, 6, 0], [3, 3, 3, 3], [6, 6, 6, 6], [5, 5, 5, 5], [2, 2, 2, 2], [4, 4, 4, 4], [1, 1, 1, 1]]
--------------------------------------------------------------------------------------
(3) * (x + t^3 + 3*t^2 + 6*t) * (x + 3*t^3 + 2*t^2 + 3*t) * (x + 3*t^3 + 6*t^2 + 3) * (x + 4*t^3 + t^2 + 4) * (x + 4*t^3 + 5*t^2 + 4*t) * (x + 6*t^3 + 4*t^2 + t) * (x + 1)^28 * (x + 6)^28 * (x + 2)^30 * (x + 5)^30 * (x + 3)^31 * (x + 4)^31

(105, 4, 184, [1, 1, 1, 1, 1, 1, 28, 28, 30, 30, 31, 31])

[1525, 600, 2275, 1075, 1800, 325, 1200, 2400, 2000, 800, 1600, 400]

[[6, 0, 3, 4], [5, 1, 5, 1], [0, 3, 4, 6], [4, 6, 0, 3], [1, 5, 1, 5], [3, 4, 6, 0], [3, 3, 3, 3], [6, 6, 6, 6], [5, 5, 5, 5], [2, 2, 2, 2], [4, 4, 4, 4], [1, 1, 1, 1]]
----------------------------------------------------------------------------

(4) * (x + t^3 + 3*t^2 + 6*t) * (x + 3*t^3 + 2*t^2 + 3*t) * (x + 3*t^3 + 6*t^2 + 3) * (x + 4*t^3 + t^2 + 4) * (x + 4*t^3 + 5*t^2 + 4*t) * (x + 6*t^3 + 4*t^2 + t) * (x + 1)^54 * (x + 6)^54 * (x + 2)^56 * (x + 5)^56 * (x + 3)^57 * (x + 4)^57

(196, 4, 340, [1, 1, 1, 1, 1, 1, 54, 54, 56, 56, 57, 57])

[1525, 600, 2275, 1075, 1800, 325, 1200, 2400, 2000, 800, 1600, 400]

[[6, 0, 3, 4], [5, 1, 5, 1], [0, 3, 4, 6], [4, 6, 0, 3], [1, 5, 1, 5], [3, 4, 6, 0], [3, 3, 3, 3], [6, 6, 6, 6], [5, 5, 5, 5], [2, 2, 2, 2], [4, 4, 4, 4], [1, 1, 1, 1]]
--------------------------------------------------------------------------------------


In [3]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=7
residue=1
bound=120
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        zerodata=[]
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        order=prime^sd
        td=totalDegreeOverField(poly,prime,sd)
        lm=logsAndMultiplicitiesOverFieldTest(poly,prime,sd)
        logs=[]
        digits=[]
        multiplicities=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            multiplicity=lm[j][1]
            digit=log.digits(prime)
            digit= padListOnRightToLengthK(digit,sd)
            logs=logs+[log]
            multiplicities=multiplicities+[multiplicity]
            digits=digits+[digit]
        print
        print (index,sd,td,multiplicities)
        print
        print logs;print;print digits
        print "--------------------------------------------------------------------------------------"

(6) * (x + t^3 + t^2 + t + 2) * (x + t^3 + 6*t^2 + 3*t + 4) * (x + 6*t^3 + t^2 + 4*t + 3) * (x + 6*t^3 + 6*t^2 + 6*t + 5)

(1, 4, 4, [1, 1, 1, 1])

[525, 75, 1275, 1725]

[[0, 5, 3, 1], [5, 3, 1, 0], [1, 0, 5, 3], [3, 1, 0, 5]]
--------------------------------------------------------------------------------------
(6) * (x + t^3 + t^2 + t + 2) * (x + t^3 + 6*t^2 + 3*t + 4) * (x + 6*t^3 + t^2 + 4*t + 3) * (x + 6*t^3 + 6*t^2 + 6*t + 5) * (x + 1)^2 * (x + 2)^2 * (x + 3)^2 * (x + 4)^2 * (x + 5)^2 * (x + 6)^2

(8, 4, 16, [1, 1, 1, 1, 2, 2, 2, 2, 2, 2])

[525, 75, 1275, 1725, 1200, 2000, 1600, 400, 800, 2400]

[[0, 5, 3, 1], [5, 3, 1, 0], [1, 0, 5, 3], [3, 1, 0, 5], [3, 3, 3, 3], [5, 5, 5, 5], [4, 4, 4, 4], [1, 1, 1, 1], [2, 2, 2, 2], [6, 6, 6, 6]]
--------------------------------------------------------------------------------------
(5) * (x + t^3 + t^2 + t + 2) * (x + t^3 + 6*t^2 + 3*t + 4) * (x + 6*t^3 + t^2 + 4*t + 3) * (x + 6*t^3 + 6*t^2 + 6*t + 5) * (x + 1)^4 * (x + 2)^4 * (x + 3)^4 * (


(120, 4, 208, [1, 1, 1, 1, 34, 34, 34, 34, 34, 34])

[525, 75, 1275, 1725, 1200, 2000, 1600, 400, 800, 2400]

[[0, 5, 3, 1], [5, 3, 1, 0], [1, 0, 5, 3], [3, 1, 0, 5], [3, 3, 3, 3], [5, 5, 5, 5], [4, 4, 4, 4], [1, 1, 1, 1], [2, 2, 2, 2], [6, 6, 6, 6]]
--------------------------------------------------------------------------------------
(x + t^3 + t^2 + t + 2) * (x + t^3 + 6*t^2 + 3*t + 4) * (x + 6*t^3 + t^2 + 4*t + 3) * (x + 6*t^3 + 6*t^2 + 6*t + 5) * (x + 1)^36 * (x + 2)^36 * (x + 3)^36 * (x + 4)^36 * (x + 5)^36 * (x + 6)^36

(127, 4, 220, [1, 1, 1, 1, 36, 36, 36, 36, 36, 36])

[525, 75, 1275, 1725, 1200, 2000, 1600, 400, 800, 2400]

[[0, 5, 3, 1], [5, 3, 1, 0], [1, 0, 5, 3], [3, 1, 0, 5], [3, 3, 3, 3], [5, 5, 5, 5], [4, 4, 4, 4], [1, 1, 1, 1], [2, 2, 2, 2], [6, 6, 6, 6]]
--------------------------------------------------------------------------------------
(4) * (x + t^3 + t^2 + t + 2) * (x + t^3 + 6*t^2 + 3*t + 4) * (x + 6*t^3 + t^2 + 4*t + 3) * (x + 6*t^3 + 6*t^2 + 6*t + 5) * (x +

In [4]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=7
residue=2
bound=120
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        zerodata=[]
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        order=prime^sd
        td=totalDegreeOverField(poly,prime,sd)
        lm=logsAndMultiplicitiesOverFieldTest(poly,prime,sd)
        logs=[]
        digits=[]
        multiplicities=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            multiplicity=lm[j][1]
            digit=log.digits(prime)
            digit= padListOnRightToLengthK(digit,sd)
            logs=logs+[log]
            multiplicities=multiplicities+[multiplicity]
            digits=digits+[digit]
        print
        print (index,sd,td,multiplicities)
        print
        print logs;print;print digits
        print "--------------------------------------------------------------------------------------"

(6) * (x + 2) * (x + 5) * (x + 2*t + 6)^2 * (x + 5*t + 1)^2

(2, 2, 6, [1, 1, 2, 2])

[40, 16, 28, 4]

[[5, 5], [2, 2], [0, 4], [4, 0]]
--------------------------------------------------------------------------------------
(3) * (x + 1)^2 * (x + 3)^2 * (x + 4)^2 * (x + 6)^2 * (x + 2*t + 6)^2 * (x + 5*t + 1)^2 * (x + 2)^3 * (x + 5)^3

(9, 2, 18, [2, 2, 2, 2, 2, 2, 3, 3])

[24, 32, 8, 48, 28, 4, 40, 16]

[[3, 3], [4, 4], [1, 1], [6, 6], [0, 4], [4, 0], [5, 5], [2, 2]]
--------------------------------------------------------------------------------------
(4) * (x + 2*t + 6)^2 * (x + 5*t + 1)^2 * (x + 1)^4 * (x + 3)^4 * (x + 4)^4 * (x + 6)^4 * (x + 2)^5 * (x + 5)^5

(16, 2, 30, [2, 2, 4, 4, 4, 4, 5, 5])

[28, 4, 24, 32, 8, 48, 40, 16]

[[0, 4], [4, 0], [3, 3], [4, 4], [1, 1], [6, 6], [5, 5], [2, 2]]
--------------------------------------------------------------------------------------
(2) * (x + 2*t + 6)^2 * (x + 5*t + 1)^2 * (x + 1)^6 * (x + 3)^6 * (x + 4)^6 * (x + 6)^6 * (x + 2)^7 * (x +

[28, 4, 24, 32, 8, 48, 40, 16]

[[0, 4], [4, 0], [3, 3], [4, 4], [1, 1], [6, 6], [5, 5], [2, 2]]
--------------------------------------------------------------------------------------
(5) * (x + 2*t + 6)^2 * (x + 5*t + 1)^2 * (x + 1)^50 * (x + 3)^50 * (x + 4)^50 * (x + 6)^50 * (x + 2)^51 * (x + 5)^51

(177, 2, 306, [2, 2, 50, 50, 50, 50, 51, 51])

[28, 4, 24, 32, 8, 48, 40, 16]

[[0, 4], [4, 0], [3, 3], [4, 4], [1, 1], [6, 6], [5, 5], [2, 2]]
--------------------------------------------------------------------------------------
(2) * (x + 2*t + 6)^2 * (x + 5*t + 1)^2 * (x + 1)^52 * (x + 3)^52 * (x + 4)^52 * (x + 6)^52 * (x + 2)^53 * (x + 5)^53

(184, 2, 318, [2, 2, 52, 52, 52, 52, 53, 53])

[28, 4, 24, 32, 8, 48, 40, 16]

[[0, 4], [4, 0], [3, 3], [4, 4], [1, 1], [6, 6], [5, 5], [2, 2]]
--------------------------------------------------------------------------------------
(4) * (x + 2*t + 6)^2 * (x + 5*t + 1)^2 * (x + 1)^54 * (x + 3)^54 * (x + 4)^54 * (x + 6)^54 * (x + 2)^55 * (x + 5)^5

In [5]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=7
residue=3
bound=120
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        zerodata=[]
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        order=prime^sd
        td=totalDegreeOverField(poly,prime,sd)
        lm=logsAndMultiplicitiesOverFieldTest(poly,prime,sd)
        logs=[]
        digits=[]
        multiplicities=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            multiplicity=lm[j][1]
            digit=log.digits(prime)
            digit= padListOnRightToLengthK(digit,sd)
            logs=logs+[log]
            multiplicities=multiplicities+[multiplicity]
            digits=digits+[digit]
        print
        print (index,sd,td,multiplicities)
        print
        print logs;print;print digits
        print "--------------------------------------------------------------------------------------"

(x + 2) * (x + 5) * (x + t) * (x + t + 3) * (x + t + 6) * (x + 6*t) * (x + 6*t + 1) * (x + 6*t + 4)

(3, 2, 8, [1, 1, 1, 1, 1, 1, 1, 1])

[40, 16, 25, 12, 7, 1, 31, 36]

[[5, 5], [2, 2], [4, 3], [5, 1], [0, 1], [1, 0], [3, 4], [1, 5]]
--------------------------------------------------------------------------------------
(4) * (x + t) * (x + t + 3) * (x + t + 6) * (x + 6*t) * (x + 6*t + 1) * (x + 6*t + 4) * (x + 1)^2 * (x + 3)^2 * (x + 4)^2 * (x + 6)^2 * (x + 2)^3 * (x + 5)^3

(10, 2, 20, [1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3])

[25, 12, 7, 1, 31, 36, 24, 32, 8, 48, 40, 16]

[[4, 3], [5, 1], [0, 1], [1, 0], [3, 4], [1, 5], [3, 3], [4, 4], [1, 1], [6, 6], [5, 5], [2, 2]]
--------------------------------------------------------------------------------------
(2) * (x + t) * (x + t + 3) * (x + t + 6) * (x + 6*t) * (x + 6*t + 1) * (x + 6*t + 4) * (x + 1)^4 * (x + 3)^4 * (x + 4)^4 * (x + 6)^4 * (x + 2)^5 * (x + 5)^5

(17, 2, 32, [1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 5, 5])

[25, 12, 7, 1, 31, 36, 24, 

(136, 2, 236, [1, 1, 1, 1, 1, 1, 38, 38, 38, 38, 39, 39])

[25, 12, 7, 1, 31, 36, 24, 32, 8, 48, 40, 16]

[[4, 3], [5, 1], [0, 1], [1, 0], [3, 4], [1, 5], [3, 3], [4, 4], [1, 1], [6, 6], [5, 5], [2, 2]]
--------------------------------------------------------------------------------------
(4) * (x + t) * (x + t + 3) * (x + t + 6) * (x + 6*t) * (x + 6*t + 1) * (x + 6*t + 4) * (x + 1)^40 * (x + 3)^40 * (x + 4)^40 * (x + 6)^40 * (x + 2)^41 * (x + 5)^41

(143, 2, 248, [1, 1, 1, 1, 1, 1, 40, 40, 40, 40, 41, 41])

[25, 12, 7, 1, 31, 36, 24, 32, 8, 48, 40, 16]

[[4, 3], [5, 1], [0, 1], [1, 0], [3, 4], [1, 5], [3, 3], [4, 4], [1, 1], [6, 6], [5, 5], [2, 2]]
--------------------------------------------------------------------------------------
(4) * (x + t) * (x + t + 3) * (x + t + 6) * (x + 6*t) * (x + 6*t + 1) * (x + 6*t + 4) * (x + 1)^42 * (x + 3)^42 * (x + 4)^42 * (x + 6)^42 * (x + 2)^43 * (x + 5)^43

(150, 2, 260, [1, 1, 1, 1, 1, 1, 42, 42, 42, 42, 43, 43])

[25, 12, 7, 1, 31, 36, 24, 32, 

In [6]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=7
residue=4
bound=120
for k in [0..len(s)-1]:
    index=s[k][0]
    if mod(index,prime)==residue:
        zerodata=[]
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        sd=leastSplittingDegree(poly,prime,bound)
        order=prime^sd
        td=totalDegreeOverField(poly,prime,sd)
        lm=logsAndMultiplicitiesOverFieldTest(poly,prime,sd)
        logs=[]
        digits=[]
        multiplicities=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            multiplicity=lm[j][1]
            digit=log.digits(prime)
            digit= padListOnRightToLengthK(digit,sd)
            logs=logs+[log]
            multiplicities=multiplicities+[multiplicity]
            digits=digits+[digit]
        print
        print (index,sd,td,multiplicities)
        print
        print logs;print;print digits
        print "--------------------------------------------------------------------------------------"

(3) * (x + 2) * (x + 5) * (x + 5*t^4 + t^3 + 5*t^2 + 4*t + 1) * (x + 2*t^4 + 6*t^3 + 2*t^2 + 3*t + 6) * (x + 4*t^5 + 6*t^4 + 3*t^3 + 4*t^2 + 2) * (x + 4*t^5 + 4*t^4 + 4*t^3 + 2*t^2 + 4*t + 2) * (x + 4*t^5 + 4*t^4 + 2*t^3 + 5*t^2 + t + 3) * (x + 3*t^5 + 3*t^4 + 5*t^3 + 2*t^2 + 6*t + 4) * (x + 3*t^5 + t^4 + 4*t^3 + 3*t^2 + 5) * (x + 3*t^5 + 3*t^4 + 3*t^3 + 5*t^2 + 3*t + 5)

(4, 6, 10, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

[98040, 39216, 100104, 41280, 81528, 53664, 88236, 29412, 22704, 112488]

[[5, 5, 5, 5, 5, 5], [2, 2, 2, 2, 2, 2], [4, 6, 5, 4, 6, 5], [1, 3, 2, 1, 3, 2], [6, 5, 4, 6, 5, 4], [2, 1, 3, 2, 1, 3], [1, 5, 1, 5, 1, 5], [5, 1, 5, 1, 5, 1], [3, 2, 1, 3, 2, 1], [5, 4, 6, 5, 4, 6]]
--------------------------------------------------------------------------------------
(3) * (x + 5*t^4 + t^3 + 5*t^2 + 4*t + 1) * (x + 2*t^4 + 6*t^3 + 2*t^2 + 3*t + 6) * (x + 4*t^5 + 6*t^4 + 3*t^3 + 4*t^2 + 2) * (x + 4*t^5 + 4*t^4 + 4*t^3 + 2*t^2 + 4*t + 2) * (x + 4*t^5 + 4*t^4 + 2*t^3 + 5*t^2 + t + 3) *


(67, 6, 118, [1, 1, 1, 1, 1, 1, 1, 1, 18, 18, 18, 18, 19, 19])

[100104, 41280, 81528, 53664, 88236, 29412, 22704, 112488, 58824, 78432, 19608, 117648, 98040, 39216]

[[4, 6, 5, 4, 6, 5], [1, 3, 2, 1, 3, 2], [6, 5, 4, 6, 5, 4], [2, 1, 3, 2, 1, 3], [1, 5, 1, 5, 1, 5], [5, 1, 5, 1, 5, 1], [3, 2, 1, 3, 2, 1], [5, 4, 6, 5, 4, 6], [3, 3, 3, 3, 3, 3], [4, 4, 4, 4, 4, 4], [1, 1, 1, 1, 1, 1], [6, 6, 6, 6, 6, 6], [5, 5, 5, 5, 5, 5], [2, 2, 2, 2, 2, 2]]
--------------------------------------------------------------------------------------
(5) * (x + 5*t^4 + t^3 + 5*t^2 + 4*t + 1) * (x + 2*t^4 + 6*t^3 + 2*t^2 + 3*t + 6) * (x + 4*t^5 + 6*t^4 + 3*t^3 + 4*t^2 + 2) * (x + 4*t^5 + 4*t^4 + 4*t^3 + 2*t^2 + 4*t + 2) * (x + 4*t^5 + 4*t^4 + 2*t^3 + 5*t^2 + t + 3) * (x + 3*t^5 + 3*t^4 + 5*t^3 + 2*t^2 + 6*t + 4) * (x + 3*t^5 + t^4 + 4*t^3 + 3*t^2 + 5) * (x + 3*t^5 + 3*t^4 + 3*t^3 + 5*t^2 + 3*t + 5) * (x + 1)^20 * (x + 3)^20 * (x + 4)^20 * (x + 6)^20 * (x + 2)^21 * (x + 5)^21

(74, 6, 130, [1, 1, 1, 1, 1, 1,


(130, 6, 226, [1, 1, 1, 1, 1, 1, 1, 1, 36, 36, 36, 36, 37, 37])

[100104, 41280, 81528, 53664, 88236, 29412, 22704, 112488, 58824, 78432, 19608, 117648, 98040, 39216]

[[4, 6, 5, 4, 6, 5], [1, 3, 2, 1, 3, 2], [6, 5, 4, 6, 5, 4], [2, 1, 3, 2, 1, 3], [1, 5, 1, 5, 1, 5], [5, 1, 5, 1, 5, 1], [3, 2, 1, 3, 2, 1], [5, 4, 6, 5, 4, 6], [3, 3, 3, 3, 3, 3], [4, 4, 4, 4, 4, 4], [1, 1, 1, 1, 1, 1], [6, 6, 6, 6, 6, 6], [5, 5, 5, 5, 5, 5], [2, 2, 2, 2, 2, 2]]
--------------------------------------------------------------------------------------
(x + 5*t^4 + t^3 + 5*t^2 + 4*t + 1) * (x + 2*t^4 + 6*t^3 + 2*t^2 + 3*t + 6) * (x + 4*t^5 + 6*t^4 + 3*t^3 + 4*t^2 + 2) * (x + 4*t^5 + 4*t^4 + 4*t^3 + 2*t^2 + 4*t + 2) * (x + 4*t^5 + 4*t^4 + 2*t^3 + 5*t^2 + t + 3) * (x + 3*t^5 + 3*t^4 + 5*t^3 + 2*t^2 + 6*t + 4) * (x + 3*t^5 + t^4 + 4*t^3 + 3*t^2 + 5) * (x + 3*t^5 + 3*t^4 + 3*t^3 + 5*t^2 + 3*t + 5) * (x + 1)^38 * (x + 3)^38 * (x + 4)^38 * (x + 6)^38 * (x + 2)^39 * (x + 5)^39

(137, 6, 238, [1, 1, 1, 1, 1, 1, 1, 

(3) * (x + 5*t^4 + t^3 + 5*t^2 + 4*t + 1) * (x + 2*t^4 + 6*t^3 + 2*t^2 + 3*t + 6) * (x + 4*t^5 + 6*t^4 + 3*t^3 + 4*t^2 + 2) * (x + 4*t^5 + 4*t^4 + 4*t^3 + 2*t^2 + 4*t + 2) * (x + 4*t^5 + 4*t^4 + 2*t^3 + 5*t^2 + t + 3) * (x + 3*t^5 + 3*t^4 + 5*t^3 + 2*t^2 + 6*t + 4) * (x + 3*t^5 + t^4 + 4*t^3 + 3*t^2 + 5) * (x + 3*t^5 + 3*t^4 + 3*t^3 + 5*t^2 + 3*t + 5) * (x + 1)^54 * (x + 3)^54 * (x + 4)^54 * (x + 6)^54 * (x + 2)^55 * (x + 5)^55

(193, 6, 334, [1, 1, 1, 1, 1, 1, 1, 1, 54, 54, 54, 54, 55, 55])

[100104, 41280, 81528, 53664, 88236, 29412, 22704, 112488, 58824, 78432, 19608, 117648, 98040, 39216]

[[4, 6, 5, 4, 6, 5], [1, 3, 2, 1, 3, 2], [6, 5, 4, 6, 5, 4], [2, 1, 3, 2, 1, 3], [1, 5, 1, 5, 1, 5], [5, 1, 5, 1, 5, 1], [3, 2, 1, 3, 2, 1], [5, 4, 6, 5, 4, 6], [3, 3, 3, 3, 3, 3], [4, 4, 4, 4, 4, 4], [1, 1, 1, 1, 1, 1], [6, 6, 6, 6, 6, 6], [5, 5, 5, 5, 5, 5], [2, 2, 2, 2, 2, 2]]
--------------------------------------------------------------------------------------
